## create a connection to the database using the library sqlite3

In [240]:
import pandas as pd
import sqlite3

In [241]:
conn = sqlite3.connect('../ex00/data/checking-logs.sqlite.sqlite copy')

## get the schema of the table test

In [242]:
pd.io.sql.read_sql("PRAGMA table_info(test);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,level_0,INTEGER,0,None,0
1,1,index,INTEGER,0,None,0
2,2,uid,TEXT,0,None,0
3,3,labname,TEXT,0,None,0
4,4,first_commit_ts,TIMESTAMP,0,None,0
5,5,first_view_ts,TIMESTAMP,0,None,0


## get only the first 10 rows of the table test to check what the table looks like

In [243]:
pd.io.sql.read_sql("SELECT * from test limit 10", conn)

,level_0,index,uid,labname,first_commit_ts,first_view_ts
0,3,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query

In [1]:
df_min = pd.io.sql.read_sql("SELECT uid, min((julianday(datetime(deadlines, 'unixepoch')) - julianday(timestamp))) * 24.0 as hours FROM (\
                   (SELECT * FROM checker) t1\
                   JOIN\
                   (SELECT labs, deadlines from deadlines\
                   WHERE labs not in ('project1')) t2\
                   ON t1.labname = t2.labs)\
                   WHERE uid LIKE 'user_%'\
                   GROUP BY uid", conn)
df_min.head()

NameError: name 'pd' is not defined

## do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [245]:
df_max = pd.io.sql.read_sql("SELECT uid, max((julianday(datetime(deadlines, 'unixepoch')) - julianday(timestamp))) * 24.0 as hours FROM (\
                   (SELECT * FROM checker) t1\
                   JOIN\
                   (SELECT labs, deadlines from deadlines\
                   WHERE labs not in ('project1')) t2\
                   ON t1.labname = t2.labs)\
                   WHERE uid LIKE 'user_%'\
                   GROUP BY uid", conn)
df_max

,uid,hours
0,user_1,175.556592
1,user_10,132.341699
2,user_11,2.884175
3,user_12,198.868676
4,user_13,135.329297
5,user_14,200.766302
6,user_15,32.529714
7,user_16,171.764244
8,user_17,81.591404
9,user_18,10.973376


## do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg

In [246]:
df_avg = pd.io.sql.read_sql("SELECT uid, avg((julianday(datetime(deadlines, 'unixepoch')) - julianday(timestamp))) * 24.0 as hours FROM (\
                   (SELECT * FROM checker) t1\
                   JOIN\
                   (SELECT labs, deadlines from deadlines\
                   WHERE labs not in ('project1')) t2\
                   ON t1.labname = t2.labs)\
                   WHERE uid LIKE 'user_%'\
                   GROUP BY uid", conn)
df_avg

,uid,hours
0,user_1,58.634224
1,user_10,92.766679
2,user_11,2.767897
3,user_12,28.077276
4,user_13,20.972196
5,user_14,194.207550
6,user_15,6.018639
7,user_16,114.586411
8,user_17,48.743378
9,user_18,5.381578


## we want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference

In [247]:
views_diff = pd.io.sql.read_sql("SELECT t11.uid, avg_diff, pageviews FROM\
                   (((SELECT uid, avg((julianday(datetime(deadlines, 'unixepoch')) - julianday(timestamp))) * 24.0 as avg_diff FROM (\
                   (SELECT * FROM checker) t1\
                   JOIN\
                   (SELECT labs, deadlines from deadlines\
                   WHERE labs not in ('project1')) t2\
                   ON t1.labname = t2.labs)\
                   WHERE uid LIKE 'user_%'\
                   GROUP BY uid) t11\
                   JOIN\
                   (select uid from test) t22\
                   on t11.uid = t22.uid) t111\
                   JOIN\
                   (SELECT uid, COUNT(uid) as pageviews\
                   FROM pageviews\
                   GROUP BY uid) t222\
                   ON t111.uid = t222.uid)\
                   group by t11.uid\
                    ", conn)
views_diff

,uid,avg_diff,pageviews
0,user_1,58.634224,28
1,user_10,92.766679,89
2,user_14,194.207550,143
3,user_17,48.743378,47
4,user_18,5.381578,3
5,user_19,47.333223,16
6,user_21,84.473952,10
7,user_25,47.024429,179
8,user_28,40.825129,149
9,user_3,105.375341,317


In [248]:
views_diff['avg_diff'].corr(views_diff['pageviews']).round(3)

0.4

In [249]:
conn.close()